In [18]:
from multiprocessing import freeze_support

import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader, Sampler
from torchvision import datasets
from torchvision import models
from torchvision.transforms import transforms
from torch.optim import Adam


import matplotlib.pyplot as plt
import numpy as np

from pathlib import Path
from PIL import *

In [2]:
# Hyperparameters.
num_epochs = 20
num_classes = 5
batch_size = 100
learning_rate = 0.001
num_of_workers = 5

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
import os
import pandas as pd
import csv

In [24]:
List=[]
with open("./train_labels.csv","r") as csvfile:
    reader = csv.reader(csvfile)
    #這裡不需要readlines
    for line in reader:
        List.append(line)
for i in range(len(List)):
    if i > 0:
        List[i][0]=List[i][0]+'.png'
data = pd.DataFrame(List[:47501],columns =['ID','Class'])
data.drop([0],inplace=True)
#data = pd.read_csv("./train_labels.csv")
labels = data.groupby("Class")
np.random.seed(10)
for i, Class in enumerate(data.Class.unique()):
    randomList = list(range(len(labels.get_group(Class))))
    np.random.shuffle(randomList)
    trainPart = labels.get_group(Class).iloc[randomList][:int(len(randomList)*0.8)]
    testPart = labels.get_group(Class).iloc[randomList][int(len(randomList)*0.8):]
    if (i == 0):
        trainAll = trainPart
        testAll = testPart
    else:
        trainAll = pd.concat([trainAll, trainPart], axis=0)
        testAll = pd.concat([testAll, testPart], axis=0)
trainAll.to_csv("./train.csv",index=False)
testAll.to_csv("./valid.csv",index=False)


In [31]:
class image_dataset():
    def __init__(self, csvFile, rootPath, transform):
        self.df = pd.read_csv(csvFile)
        self.rootPath = rootPath
        self.xTrain = self.df['ID']
        self.yTrain = self.df['Class']
        self.transform = transform

    def __getitem__(self, index):
        img = Image.open(os.path.join(self.rootPath, self.xTrain[index]))
        img = img.convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        return img, self.yTrain[index]

    def __len__(self):
        return len(self.xTrain.index)
    
    def __class__(self):
        return sorted(self.df.label.unique())

In [127]:
inputSize = 224
dataTransformsTrain = transforms.Compose([
     transforms.RandomResizedCrop(inputSize),
     transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

dataTransformsTest = transforms.Compose([
     transforms.Resize(inputSize),
     transforms.CenterCrop(inputSize),
     transforms.ToTensor(),
     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

trainDatasets = image_dataset("./train.csv", "./train", dataTransformsTrain)
testDatasets = image_dataset("./valid.csv", "./train", dataTransformsTest)
train_loader = torch.utils.data.DataLoader(trainDatasets, batch_size=32, shuffle=True, num_workers=1)
test_loader= torch.utils.data.DataLoader(testDatasets, batch_size=32, shuffle=False)

# model = VGG16_pretrained_model(numClasses=7, featureExtract=True, usePretrained=True).to(device)
    


In [128]:
# Create model, optimizer and loss function
#model = CNNet(num_classes)

model = models.resnet18(pretrained=True)
fc_features = model.fc.in_features
model.fc = nn.Linear(fc_features, 10)

# if cuda is available, move the model to the GPU

model.cuda()
# Define the optimizer and loss function
optimizer = Adam(model.parameters(), lr=0.0001, weight_decay=0.0001)
loss_fn = nn.CrossEntropyLoss()

In [129]:
def save_models(epoch):
    torch.save(model.state_dict(), f"{epoch}.model")
    print("Checkpoint saved")

In [130]:
def test():
    model.eval()
    test_acc = 0.0
    count=0
    for x, label in test_loader:
        x = x.to(device)
        label = label.to(device, dtype=torch.long)
        output = model(x)

        # Predict classes using images from the test set
        _, prediction = torch.max(output.data, 1)

        test_acc += torch.sum(prediction == label).float()

    # Compute the average acc and loss over all 10000 test images
    test_acc = test_acc /  10000* 100
    return test_acc

In [133]:
def train(num_epoch):
    best_acc = 0.0

    for epoch in range(num_epoch):
        model.train()
        train_acc = 0.0
        train_loss = 0.0
        count=0
        for x, label in train_loader:
            count+=1
            if (count%1000==0):
                print(count)
            x = x.to(device)
            label = label.to(device, dtype=torch.long)
            optimizer.zero_grad()
            output = model(x)
            loss = loss_fn(output, label)
            loss.backward()
            optimizer.step()
            #train_loss += loss.cpu().data[0] * images.size(0)
            train_loss += loss.item()
            _, prediction = torch.max(output.data, 1)
           

            train_acc += torch.sum(prediction == label).float()

        # Call the learning rate adjustment function
        #adjust_learning_rate(epoch)

        # Compute the average acc and loss over all 50000 training images
        train_acc = train_acc / 40000 * 100
        train_loss = train_loss / 80000

        # Evaluate on the test set
        test_acc = test()

        # Save the model if the test acc is greater than our current best
        if train_acc > best_acc:
            save_models(epoch)
            best_acc = train_acc

        # Print the metrics
        print(f"Epoch {epoch + 1}, Train Accuracy: {train_acc} , TrainLoss: {train_loss} , Test Accuracy: {test_acc}")

In [ ]:
if __name__ == '__main__':
    freeze_support()
    train(num_epochs)

1000
Checkpoint saved
Epoch 1, Train Accuracy: 68.61249542236328 , TrainLoss: 0.011864262460172177 , Test Accuracy: 84.87999725341797
1000
Checkpoint saved
Epoch 2, Train Accuracy: 74.17250061035156 , TrainLoss: 0.00936572665348649 , Test Accuracy: 85.8699951171875
1000
Checkpoint saved
Epoch 3, Train Accuracy: 76.0 , TrainLoss: 0.008593115441687406 , Test Accuracy: 86.07999420166016
1000
Checkpoint saved
Epoch 4, Train Accuracy: 77.43000030517578 , TrainLoss: 0.007955341605376452 , Test Accuracy: 86.91999816894531
1000
Checkpoint saved
Epoch 5, Train Accuracy: 78.69499969482422 , TrainLoss: 0.007472392782755196 , Test Accuracy: 88.29999542236328
1000
Checkpoint saved
Epoch 6, Train Accuracy: 79.12249755859375 , TrainLoss: 0.007150246860273182 , Test Accuracy: 88.72000122070312
1000
Checkpoint saved
Epoch 7, Train Accuracy: 79.49500274658203 , TrainLoss: 0.006968290718598291 , Test Accuracy: 88.72000122070312
1000
Checkpoint saved
Epoch 8, Train Accuracy: 79.70249938964844 , TrainLoss: